In [1]:
import json
import os

In [2]:
ast_file = os.path.join(os.getcwd(),'ast_dump.json')

In [10]:
with open(ast_file, 'r') as f:
    ast_dict = json.load(f)

len(ast_dict['ext'])

3

In [23]:
 # true_block = node['iftrue']
    # true_block_calls = []
    # num_exec_path_in_true_block = 1

    # for item in true_block['block_items']:
    #     if item['_nodetype'] == 'If':
    #         # Things are a bit complicated here. Need help simplifying 
    #         if_else_paths = parse_if_else_node(item)
    #         if len(if_else_paths) == 0: # This means there are no function call inside the if else block.
    #             pass
    #         elif len(if_else_paths)>0 and isinstance(if_else_paths[0], list):
    #             # Means multiple paths found. Clone and Append
    #             pass
    #         elif len(if_else_paths)>0 and isinstance(if_else_paths[0], str):
    #             # Means a monolithic path with one or two function call. No jump inside
    #             pass
    #         else:
    #             pass

    #     elif item['_nodetype'] == 'FuncCall':
    #         if num_exec_path_in_true_block == 1:
    #             true_block_calls.extend(parse_func_call_node(item))
    #         else:
    #             for i in range(num_exec_path_in_true_block):
    #                 true_block_calls[i].extend(parse_func_call_node(item))
    #     elif item['_nodetype'] == 'Assignment':
    #         #parse_assignment_node(node)
    #         if num_exec_path_in_true_block == 1:
    #             true_block_calls.extend(parse_assignment_node(item))
    #         else:
    #             for i in range(num_exec_path_in_true_block):
    #                 true_block_calls[i].extend(parse_assignment_node(item))

    #     else:
    #         pass

    # false_block = node['iffalse']
    # false_block_calls = []

In [13]:
for element in ast_dict['ext']:
    if element['_nodetype'] == 'FuncDef':
        print("A function definiton {}".format(element['decl']['name']))
        for item in element['body']['block_items']:
            if item['_nodetype'] == 'FuncCall':
                print("Found Function call of {}".format(item['name']['name']))

A function definiton sumfunc
Found Function call of printf
A function definiton main
Found Function call of somefunc


In [2]:
class SimplifiedAstNode:
    def __init__(self, _type, _name = None):
        self.type = _type
        self.name = _name # Only has a name if node_type is FuncCall
        self.calls = [] # Only has calls if node_type is if-else and FuncCalls are in condition blocks
        self.children = []
        self.children_json = []
    

In [8]:
def parse_func_call_node(root, node, is_if_node = False):
    assert node['_nodetype'] == 'FuncCall'
    print("Function Call Node Found")
    func_call = node['name']['name']
    temp_node = SimplifiedAstNode('FuncCall', func_call)
    if is_if_node:
        root.calls.append(temp_node)
        print("Function call inside if caluse: {}".format(func_call))
    else:
        root.children.append(temp_node)
        print("Function Call: {}".format(func_call))
    

def parse_assignment_node(root, node):
    assert node['_nodetype'] == 'Assignment'
    print("Assignment Node Found")
    if node['rvalue']['_nodetype'] == 'FuncCall':
        parse_func_call_node(root, node['rvalue'])
    

def parse_if_else_node(root,node):
    assert node['_nodetype'] == 'If'
    print("If Node Found")

    curr_node = SimplifiedAstNode('If')
    root.children.append(curr_node)
    
    if node['cond']['left']['_nodetype'] == 'FuncCall':
        # Do something
        parse_func_call_node(curr_node, node['cond']['left'], is_if_node=True)
        
    if node['cond']['right']['_nodetype'] == 'FuncCall':
        # Do something
        parse_func_call_node(curr_node, node['cond']['right'], is_if_node=True)
        
    
    true_node = SimplifiedAstNode('iftrue')
    curr_node.children.append(true_node)
    if 'iftrue' in node:
        true_block = node['iftrue']
        if 'block_items' in true_block and true_block['block_items'] is not None:
            for item in true_block['block_items']:
                if item['_nodetype'] == 'If':
                    parse_if_else_node(true_node, item)
                elif item['_nodetype'] == 'FuncCall':
                    parse_func_call_node(true_node, item)            
                elif item['_nodetype'] == 'Assignment':
                    parse_assignment_node(true_node, item)
                else:
                    pass

    false_node = SimplifiedAstNode('iffalse')
    curr_node.children.append(false_node)
    if 'iffalse' in node and node['iffalse'] is not None:
        false_block = node['iffalse']
        if 'block_items' in false_block and false_block['block_items'] is not None:
            for item in false_block['block_items']:
                if item['_nodetype'] == 'If':
                    parse_if_else_node(false_node, item)
                elif item['_nodetype'] == 'FuncCall':
                    parse_func_call_node(false_node, item)            
                elif item['_nodetype'] == 'Assignment':
                    parse_assignment_node(true_node, item)
                else:
                    pass
        

In [9]:
def parse_function_definition(node):
    assert node["_nodetype"] == 'FuncDef'
    items_in_function_block = node["body"]["block_items"]
    function_name = node["decl"]["name"]
    function_root_node = SimplifiedAstNode('FuncDef', function_name)
    num_block_items = len(items_in_function_block)

    for item in items_in_function_block:
        if item['_nodetype'] == 'FuncCall':
            parse_func_call_node(function_root_node, item)
        elif item['_nodetype'] == 'Assignment':
            parse_assignment_node(function_root_node, item)    
        elif item['_nodetype'] == 'If':
            parse_if_else_node(function_root_node, item)
        else:
            pass

    return function_root_node

In [22]:
test_file_path = os.path.join(os.getcwd(),'ast_dump.json')

In [23]:
with open(test_file_path, 'r') as f:
    ast_dict = json.load(f)

len(ast_dict['ext'])

4

In [24]:
curr_root = None
for element in ast_dict['ext']:
    if element['_nodetype'] == 'FuncDef':
        print("A function definiton {}".format(element['decl']['name']))
        curr_root = parse_function_definition(element)

A function definiton a_random_stuff
A function definiton sumfunc
Function Call Node Found
Function Call: printf
A function definiton main
Assignment Node Found
Function Call Node Found
Function Call: somefunc
Assignment Node Found
Function Call Node Found
Function Call: fopen
If Node Found
Function Call Node Found
Function call inside if caluse: fopen
Function Call Node Found
Function Call: printf
Function Call Node Found
Function Call: fclose
Function Call Node Found
Function Call: strcpy
Function Call Node Found
Function Call: printf


In [17]:
from queue import Queue

In [18]:
def get_paths_from_conditon_node(node):
    paths = []
    if len(node.calls)>0:
        paths.extend(node.calls)
    true_node = node.children[0]
    false_node = node.children[1]

    true_paths = get_paths_from_branch(true_node)
    # print("True node length: {}".format(len(true_paths)))
    false_paths = get_paths_from_branch(false_node)
    # if false_paths is not None:
    #     print("False node length: {}".format(len(false_paths)))
    
    num_paths = 0
    if false_paths is not None:
        num_paths += len(false_paths)
    
    num_paths += len(true_paths)

    result_paths = []
    if len(paths) == 0:
        result_paths.extend(true_paths)
        if false_paths is not None:
            result_paths.extend(false_paths)
    else:
        for path in paths:
            for t_path in true_paths:
                temp = path.copy()
                temp.extend(t_path)
                result_paths.append(temp)
            if false_paths is not None:
                for f_path in false_paths:
                    temp = path.copy()
                    temp.extend(f_path)
                    result_paths.append(temp)
    
    return result_paths



def get_paths_from_branch(node):
    if node is None:
        return None
    paths = list() # [Cal3, Call4]
    paths.append([])
    for child in node.children:
        if child.type == 'FuncCall':
            for path in paths:
                path.append(child.name)
        elif child.type == 'Assignment':
            for path in paths:
                path.append(child.name)
        elif child.type == 'If':
            temp_paths = get_paths_from_conditon_node(child) # [[], [Call5]]
            new_paths = []
            for path in paths:
                for t_path in temp_paths:
                    temp_original_path = path.copy()
                    temp_original_path.extend(t_path)
                    new_paths.append(temp_original_path)
            paths = new_paths
        else:
            pass
    
    return paths

def get_paths_from_func_def(node):
    block_paths = []
    for child in node.children:
        if child.type == 'FuncCall':
            block_paths.append([child.name])
        elif child.type == 'Assignment':
            block_paths.append([child.name])
        elif child.type == 'If':
            block_paths.append(get_paths_from_conditon_node(child))

    result_paths = []
    q = Queue()
    q.put([])

    for block_path in block_paths:
        curr_size = q.qsize()
        for i in range(curr_size):
            incomplete_path = q.get()
            if len(block_path) == 1:
                temp = incomplete_path.copy()
                temp.extend(block_path)
                q.put(temp)
            elif len(block_path)>1:
                for path in block_path:
                    temp = incomplete_path.copy()
                    temp.extend(path)
                    q.put(temp)
    
    while q.empty() == False:
        result_paths.append(q.get())
    
    return result_paths
    





In [19]:
curr_root.children[2].type

'If'

In [20]:
paths = get_paths_from_func_def(curr_root)
#paths = get_paths_from_conditon_node(curr_root.children[2])

In [21]:
print(paths)

[['A', 'B', 'printf', 'printf'], ['A', 'B', 'printf'], ['A', 'B', 'exit']]


In [1]:
syscall_file_name = '/home/rashik/Documents/glib_source/glibc-2.36/sysdeps/unix/syscalls.list'

In [3]:
syscall_map = os.path.join(os.getcwd(), 'syscall_map.json')

In [4]:
line = 'recvfrom	-	recvfrom	Ci:ibUiBN	__libc_recvfrom __recvfrom recvfrom'
tokens = line.split()
for token in tokens:
    print(token)

recvfrom
-
recvfrom
Ci:ibUiBN
__libc_recvfrom
__recvfrom
recvfrom


In [6]:
import json

In [7]:
sysmap = dict()
with open(syscall_file_name, 'r') as f:
    for line in f:
        tokens = line.split()
        syscall_name = tokens[2]
        for i in range(4, len(tokens),1):
            sysmap[tokens[i]] = syscall_name


json.dump(sysmap, open(syscall_map, 'w'))